In [1]:
#|default_exp tensor_utils
#|export
import torch

# 张量网络的图形表示


*   使用一个块来表示**张量**，用连接该块的**线条**代表该**张量**的**指标**。

*   每条线段被限制为只能连接一个或两个张量，当线段连接两个张量时，对应的指标为共有指标，需被收缩（求和）掉。

*   张量、指标较多时，图形比公式更直观好用。


## 例子

(a) $X_{abc} = \sum_{ijk} A_{iaj} B_{jbk} C_{kci}$

(b) $\varphi_{s_0 s_1 \dots s_{N-1}} = \sum_{\{a_\ast\}} \prod_{n=0}^{N-1} A_{a_n s_n a_{n+1}}^{(n)} I_{a_0 a_N}$

叫$N$矩阵乘积态，图例是 $N=5$

(c) $T_{abc} = \sum_{n_0 n_1 n_2 n_3} \Upsilon_{n_0} U_{a n_1} V_{b n_2} W_{c n_3} I_{n_0 n_1 n_2 n_3}$

![tensor_network_examples](images/tensor_network_examples.png)

### 注意点： Index 顺序

图像里一般不会标注 index 的顺序，这个默认是作者自己知道的。

如果一定需要标准 index 顺序，可以在靠近张量的地方标注 index 的顺序，但是不要标在线段上，因为公有的 index 的顺序在共享的张量上不一定一样。

### 注意点：单位矩阵的作用

(b) $\varphi_{s_0 s_1 \dots s_{N-1}} = \sum_{\{a_\ast\}} \prod_{n=0}^{N-1} A_{a_n s_n a_{n+1}}^{(n)} I_{a_0 a_N}$

里面的 $I_{a_0 a_N}$ 的意思是 $a_0$ 和 $a_N$ 是共享（同一个）index

同理 (c) $T_{abc} = \sum_{n_0 n_1 n_2 n_3} \Upsilon_{n_0} U_{a n_1} V_{b n_2} W_{c n_3} I_{n_0 n_1 n_2 n_3}$ 里的 $I_{n_0 n_1 n_2 n_3}$ 表示 $n_0, n_1, n_2, n_3$ 共享（同一个）index

原因：
* 为了公式简洁，比如 (b) 中就不用单独把 $a_0, a_N$ 单独命名一个共有 index 了
* 图像上，例如 (c) 中 $n_0, n_1, n_2, n_3$ 共享（同一个）index，但是不能画出来，所以添加一个单位矩阵 $I_{n_0 n_1 n_2 n_3}$

## 超单位张量

![identity_tensor](images/identity_tensor.png)

进一步解释了单位矩阵的作用，因为只有$s_0 = s_1 = \cdots = s_{K-1}$ 的时候，相乘才有贡献

In [2]:
U = torch.randn(2, 2)
V = torch.randn(2, 2)
W = torch.randn(2, 2)
gamma = torch.randn(2)

T1 = torch.einsum("n,an,bn,cn->abc", gamma, U, V, W)

I = torch.zeros(2, 2, 2, 2, dtype=torch.float32)
I[0, 0, 0, 0] = 1.
I[1, 1, 1, 1] = 1.

T2 = torch.einsum('i,aj,bk,cl,ijkl->abc', gamma, U, V, W, I)

assert T1.allclose(T2)

In [ ]:
#|export
def identity_tensor(order: int, dim: int, dtype: torch.dtype = torch.float32) -> torch.Tensor:
    """
    Create an identity tensor of given order and dimension.
    Args:
        order (int): The order of the tensor.
        dim (int): The dimension of the tensor.
        dtype (torch.dtype): The data type of the tensor. Default is torch.float32.
    Returns:
        torch.Tensor: The identity tensor of shape (dim, dim, ..., dim) with the specified order.
    """
    dims = [dim] * order
    I = torch.zeros(*dims, dtype=dtype)
    for i in range(dim):
        indices = [i] * order
        I[tuple(indices)] = 1.

    return I